<a ><img src="https://qarlab.de/wp-content/uploads/2021/02/qar-lab_logo-web1.png"  width="120" align="right"> </a>

<a ><img src="https://milani.ch/assets/Logo/Trumpf-logo.svg"  width="90" align="left"> </a>


<br>
<br>
<br>
<br>
<br>
<br>
<br>

<h1><center>Parameter Optimization </center></h1>



---
---
---

### Imports and Basics

In [1]:
import dimod
import numpy as np
import pandas as pd

from qubo_util import *
from dwave_qbsolv import QBSolv

In [2]:
### Lets define the basics

bend = [0, 1, 2]
weld = [3, 4]
paint =  [5]

bend_length = 2
weld_length = 3
paint_length = 6

t_step = 3
t_step_in_sec = 30

In [3]:
jobs = [(0, 0, 1, 2, 1, 3),
 (1, 0, 0, 2, 0, 1)]
operations = job_info(jobs, bend_length, weld_length, paint_length, t_step_in_sec, bend, weld, paint, t_step)

The maximal maketime for the given operations is: 420 second(s).
Which is equal to: 14 time steps.


Anzahl an Kombinationen: 12

   order  part  bend  weld  paint  deadline
0      0     0     2     6      6        18
1      1     0     0     6      0         6


### Initial Parameter:  $\alpha, \beta, \gamma, \delta$

Based on the (logical) Importance of the components of our Hamiltonian:

- $\alpha$ at $2$ because its a *hard* constraint - it must not be broken
 
- $\beta$ at $1.5$ because its a *soft* constraint - it should not be broken
 
- $\gamma$ at $2$ because its a *hard* constraint - it must not be broken
 
- $\delta$ at $0.035$ so it has an effect in the same realm as the constraints
 

### To verify our Parameters are robust, we implement a grid search in every direction

In [4]:
df = pd.DataFrame(columns=["alpha", "beta", "gamma", "delta", "H0", "H1", "H2", "H3"])
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'OMG'})

In [5]:
pd.set_option('display.max_rows', 1400)

In [6]:
# PARAMETER OPTIMIZATION

In [7]:
#QBSolve
def parameter_normalization(dff, jobs, alpha, beta, gamma, delta):
    
    for i in range(1):
        QUBO = np.zeros((len(operations), len(operations)))
        QUBO = h0_constraint(QUBO, operations, bend, weld, paint, alpha)
        QUBO = h1_constraint(QUBO, operations, bend, weld, paint, bend_length, weld_length, beta)
        QUBO = h2_constraint(QUBO, operations, weld, paint, bend_length, weld_length, paint_length, gamma)
        QUBO = h3_constraint(QUBO, jobs, operations, weld, paint, bend_length, weld_length, paint_length, delta)
        
        qubo_dictionary = qubo_to_dictionary_ohne_null(QUBO, operations)
    
        result = QBSolv().sample_qubo(qubo_dictionary)
        # Erste Sample
        schedule = get_results(result, operations, 0)
        
        h0 = str(h0_constraint_is_fulfilled(schedule, jobs))
        h1 = str(h1_constraint_is_fulfilled(schedule, paint_length, bend_length, weld_length))
        h2 = str(h2_constraint_is_fulfilled(schedule, paint_length, bend_length, weld_length))
        h3 = str(h3_constraint_is_fulfilled(schedule, jobs, paint_length, bend_length, weld_length))
        
        
        df2 = pd.DataFrame([[alpha, beta, gamma, delta, h0, h1, h2, h3]], columns=["alpha", "beta", "gamma", "delta", "H0", "H1", "H2", "H3"])
        #Wenn nicht 100% von den Erbebnissen sind richtig, append to dff
        #if (len(set( result.data_vectors['energy'])) != 1):
        # Just to see the complete Dataframe, we removed the check --> with check the DF ist empty because all constraints are fullfilled
        dff = dff.append(df2)
        return dff
    

### For all directions, check for energy =! max Energy

In [8]:
for a in range (150, 250, 20):
    for b in range (100, 200, 20):
        for g in range (150, 250, 20):
            for d in range (30, 40, 2):
                df = parameter_normalization(df, jobs, a, b, g, d/10)
df

,OMG,alpha,beta,gamma,delta,H0,H1,H2,H3
0,NaN,150,100,150,3.0,True,True,True,True
0,NaN,150,100,150,3.2,True,True,True,True
0,NaN,150,100,150,3.4,True,True,True,True
0,NaN,150,100,150,3.6,True,True,True,True
0,NaN,150,100,150,3.8,True,True,True,True
0,NaN,150,100,170,3.0,True,True,True,True
0,NaN,150,100,170,3.2,True,True,True,True
0,NaN,150,100,170,3.4,True,True,True,True
0,NaN,150,100,170,3.6,True,True,True,True
0,NaN,150,100,170,3.8,True,True,True,True


---

## --> Therefore our Parameters are robust and can be used for all our job sets